In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/A_Z Handwritten Data/A_Z Handwritten Data.csv


In [2]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras import backend as K
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

Using TensorFlow backend.


In [3]:
data = pd.read_csv('/kaggle/input/A_Z Handwritten Data/A_Z Handwritten Data.csv')

In [4]:
data.rename(columns = {'0': 'labels'}, inplace = True)

In [5]:
X = data.drop('labels', axis = 1)
Y = data['labels']
X.shape, Y.shape

((372450, 784), (372450,))

In [6]:
(xTrain, xTest, yTrain, yTest) = train_test_split(X, Y)

In [7]:
scaler = MinMaxScaler()
scaler.fit(xTrain)

MinMaxScaler(copy=True, feature_range=(0, 1))

In [8]:
xTrain = scaler.transform(xTrain)
xTest = scaler.transform(xTest)

In [16]:
xTest.shape, xTrain.shape

((93113, 28, 28, 1), (279337, 784))

In [18]:
xTrain = xTrain.reshape((xTrain.shape[0], 28, 28, 1)).astype('float32')
xTest = xTest.reshape((xTest.shape[0], 28, 28, 1)).astype('float32')
XTrain.shape, xTest.shape, yTrain.shape

((279337, 28, 28, 1), (93113, 28, 28, 1), (279337, 26))

In [11]:
yTrain = np_utils.to_categorical(yTrain)
yTest = np_utils.to_categorical(yTest)
yTrain.shape, yTest.shape

((279337, 26), (93113, 26))

In [12]:
numClasses = yTrain.shape[1]
numClasses

26

In [20]:
model = Sequential()

model.add(Conv2D(64, (5, 5), input_shape = (28, 28, 1), activation = 'relu', data_format = 'channels_last', padding = 'same'))
model.add(Conv2D(64, (5, 5), input_shape = (28, 28, 1), activation = 'relu', data_format = 'channels_last', padding = 'same'))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Conv2D(128, (3, 3), activation = 'relu', data_format = 'channels_last', padding = 'same'))
model.add(Conv2D(128, (3, 3), activation = 'relu', data_format = 'channels_last', padding = 'same'))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dense(128, activation = 'relu'))
model.add(Dense(128, activation = 'relu'))
model.add(Dense(numClasses, activation = 'softmax'))

model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])


In [21]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 28, 28, 64)        1664      
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 28, 28, 64)        102464    
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 14, 14, 128)       73856     
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 14, 14, 128)       147584    
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 7, 7, 128)         0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 7, 7, 128)         0         
__________

In [22]:
model.fit(xTrain, yTrain, validation_data=(xTest, yTest), epochs=20, batch_size=256, verbose=2)

Train on 279337 samples, validate on 93113 samples
Epoch 1/20
 - 30s - loss: 0.2159 - acc: 0.9408 - val_loss: 0.0685 - val_acc: 0.9806
Epoch 2/20
 - 29s - loss: 0.0514 - acc: 0.9856 - val_loss: 0.0475 - val_acc: 0.9874
Epoch 3/20
 - 29s - loss: 0.0380 - acc: 0.9891 - val_loss: 0.0420 - val_acc: 0.9885
Epoch 4/20
 - 29s - loss: 0.0320 - acc: 0.9905 - val_loss: 0.0361 - val_acc: 0.9910
Epoch 5/20
 - 29s - loss: 0.0255 - acc: 0.9924 - val_loss: 0.0364 - val_acc: 0.9901
Epoch 6/20
 - 29s - loss: 0.0215 - acc: 0.9934 - val_loss: 0.0330 - val_acc: 0.9917
Epoch 7/20
 - 29s - loss: 0.0181 - acc: 0.9943 - val_loss: 0.0351 - val_acc: 0.9906
Epoch 8/20
 - 29s - loss: 0.0159 - acc: 0.9950 - val_loss: 0.0282 - val_acc: 0.9930
Epoch 9/20
 - 29s - loss: 0.0148 - acc: 0.9953 - val_loss: 0.0299 - val_acc: 0.9930
Epoch 10/20
 - 29s - loss: 0.0126 - acc: 0.9959 - val_loss: 0.0318 - val_acc: 0.9924
Epoch 11/20
 - 29s - loss: 0.0117 - acc: 0.9963 - val_loss: 0.0338 - val_acc: 0.9931
Epoch 12/20
 - 29s - lo

In [23]:
model.save('weights.model')
model.save_weights("model.h5")
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
